### Append columns to Exploits DB using Solace Relativity lookup tables

In [4]:
import pandas as pd
import json
import numpy as np
import math

## Load Exploits, Protocols(tbd), and lookup tables (transactional and temporal) 
exploitsDf = pd.read_csv('../protocols/exploits.csv')
#protocolsDf = pd.read_csv('../data/protocols.csv')
f = open('../governance/lookupTablesSolace.json')
lookupTables = json.loads(f.read())
f.close()

# Create dfs for lookup tables
tvlSolaceLookupTable = pd.DataFrame(lookupTables['tvlSolaceLookup'])
auditsSolaceLookupTable = pd.DataFrame(lookupTables['auditSolaceLookup'])
oraclesSolaceLookupTable = lookupTables['oracleSolaceLookup']

# Calc Exploit Damage ratio
exploitsDf['tvlPreExploit'] = exploitsDf['tvlPreExploit'] / 10
exploitsDf['exploitDamage'] = exploitsDf['loss'] / exploitsDf['tvlPreExploit']
exploitsDf['exploitDamage']= exploitsDf['exploitDamage'].astype(object).replace(np.nan, 'null')

# Calc time to exploit
exploitsDf[['launchDate','exploitDate']] = exploitsDf[['launchDate','exploitDate']].apply(pd.to_datetime) #if conversion required
exploitsDf['timeToExploit'] = (exploitsDf['exploitDate'] - exploitsDf['launchDate']).dt.days
exploitsDf['timeToExploit'] = exploitsDf['timeToExploit'].astype(object).replace(np.nan, 'null')

# Derive tvlSolace column
def lookup_tvlSolace(tvl):
    if math.isnan(tvl):
        return 'null'
    match = (tvlSolaceLookupTable['lowerLimit'] <= tvl) & (tvlSolaceLookupTable['upperLimit'] > tvl)
    tvlOut = tvlSolaceLookupTable['tvlSolace'][match]
    return tvlOut.values[0]
exploitsDf['tvlSolace'] = exploitsDf['tvlPreExploit'].apply(lookup_tvlSolace)

# Derive auditSolace column
def lookup_auditSolace(audits):
    if math.isnan(audits):
        return 'Unknown'
    match = (auditsSolaceLookupTable['auditsOg'] == audits)
    auditsOut = auditsSolaceLookupTable['auditSolace'][match]
    return auditsOut.values[0]
exploitsDf['auditSolace'] = exploitsDf['audits'].apply(lookup_auditSolace)

# Derive oracleSolace column
def lookup_oracleSolace(oracles,catalog):
    if pd.isna(catalog):
        return 'Unknown'
    elif pd.isna(oracles):
        return 'No'
    else:
        return "Yes"
exploitsDf['oracleSolace'] = exploitsDf.apply(lambda x: lookup_oracleSolace(x.oracles, x.category), axis=1)

# TODO - Add in tag/category of the columns to exploitsDf IF needed
#      - Process the Protocols data cleansing and enrichment pipeline
#      - Run the relatiity function
#      - Run publishSeries.py to publish the data
print(exploitsDf.head(40))


          name    solaceID        appId  \
0    Akropolis   akropolis          NaN   
1     Alchemix    alchemix     alchemix   
2        Alpha       alpha     alpha-v2   
3      Anyswap     anyswap          NaN   
4    Aperocket   aperocket          NaN   
5    Autoshark   autoshark          NaN   
6     Balancer    balancer  balancer-v2   
7       Bancor      bancor       bancor   
8        Bearn       bearn          NaN   
9         Belt        belt          NaN   
10    Blizzard    blizzard          NaN   
11      Bogged      bogged          NaN   
12      Bondly      bondly          NaN   
13         Boy         boy          NaN   
14  Bt.finance  bt.finance          NaN   
15  Burgerswap  burgerswap          NaN   
16         Bzx         bzx          NaN   
17      Casper      casper          NaN   
18      Cheese      cheese          NaN   
19    Compound    compound     compound   
20       Cover       cover          NaN   
21       Cream       cream        cream   
22         